# The interpretations using LIME

# Heart dataset

In [11]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime as dt
import sys
sys.path.append('./explanation_helper')
import expla_helper as explanation

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

print(heart.dtypes)

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
target        int64
dtype: object


# Wine dataset

In [12]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime as dt
import sys
sys.path.append('./explanation_helper')
import expla_helper as explanation


wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)

wine = explanation.to_float(wine)

print(wine.dtypes)

Pregnancies                 float64
Glucose                     float64
BloodPressure               float64
SkinThickness               float64
Insulin                     float64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                         float64
target                        int64
dtype: object


# Diabetes dataset

In [13]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime as dt

import sys
sys.path.append('./explanation_helper')
import expla_helper as explanation


diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)

diabetes = explanation.to_float(diabetes)

print(diabetes.dtypes)


fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
target                    int64
dtype: object


# LIME 

## Choosing 4 instances (TP, FP, TN, FN) of each dataset randomly from DT to be our baseline

Code to choose 4 instances (TP, FP, TN, FN) of each dataset randomly, we have choosed the decision tree as the baseline classificator to choose the confusion matrix metrics

### Get the samples from the baseline

In [14]:
import numpy as np
import pandas as pd
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

np.random.seed(1)

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
model = serializer.load_model(file_name = 'serialized_model/DT_heart.pkl')
heart_sample = explanation.sample_baseline_model(model, heart)

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
model = serializer.load_model(file_name = 'serialized_model/DT_wine.pkl')
wine_sample = explanation.sample_baseline_model(model, wine)

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
model = serializer.load_model(file_name = 'serialized_model/DT_diabetes.pkl')
diabetes_sample = explanation.sample_baseline_model(model, diabetes)


# Parâmetros LIME

In [15]:
import pandas as pd
import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
from sklearn.model_selection import train_test_split
import math


# kernel_width = [math.ceil(dataset.shape[1]*0.1),
#                 math.ceil(dataset.shape[1]*0.3),
#                 math.ceil(dataset.shape[1]*0.5),
#                 math.ceil(dataset.shape[1]*0.7)]

feature_selection = ['forward_selection', 'lasso_path', 'none', 'auto']
discretize_continuous = [True]
discretizer = ['quartile', 'decile']
sample_around_instance = [True, False]

## MLP

### Heart dataset - explanation

In [16]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/MLP_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sa

serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_expl

serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/MLP_heart_instanc

serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/MLP_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/MLP_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/MLP_hear

### Wine dataset - explanation

In [17]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/MLP_wine.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)

serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s

serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/MLP_wine_instance:291

serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/MLP_wine_instance:725_num_features:8_feat_sel

serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/MLP_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/MLP_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/MLP_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/MLP_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/MLP_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:7.html
serialized_expla

### Diabetes dataset

In [18]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/MLP_diabetes.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_

serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartil

serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_ke

serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/MLP_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:9.html

## Random Forest

### Heart dataset

In [19]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/RF_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_

serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/

serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/RF_heart_instance:143_num_featu

serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/RF_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/RF_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/RF_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/RF_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:10.html
serialized_ex

### Wine dataset

In [20]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/RF_wine.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:1.html


serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/RF_wine_instance:291_num_feat

serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/RF_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/RF_wine_instance:725_num_features:8_feat

serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/RF_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/RF_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/RF_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/RF_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/RF_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/RF_wine_instance:725_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/RF_wine_instance:725_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:7.html
serialized_e

### Diabetes dataset

In [21]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/RF_diabetes.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:d

serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:4

serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:False_kernel_s:6.html
serialized_explanati

serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:auto_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/RF_diabetes_instance:86_num_features:11_feat_selec:auto_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/RF_diabet

## Naive Bayes

### Heart dataset

In [22]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/NaiveBayes_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_heart_instance:14

serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/NaiveBayes_heart_instance:143_num_

serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:1

serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/NaiveBayes_heart_instance:143_num_features:13_

serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/NaiveBayes_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:10.html


### Wine dataset

In [23]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/NaiveBayes_wine.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:

serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_s

serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:none_disc

serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/NaiveBayes_wine_instance:291_num_features:8_feat_selec:none_discre:decile_

### Diabetes dataset

In [24]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/NaiveBayes_diabetes.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_dia

serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_inst

serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_f

serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:none_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/NaiveBayes_diabetes_instance:86_num_features:11_feat_selec:fo

serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:none_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/NaiveBayes_diabetes_instance:1003_num_features:11_feat_selec:auto_disc

## Decision Tree

### Heart dataset

In [25]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/DT_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_

serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/

serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/DT_heart_instance:143_num_featu

serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/DT_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/DT_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/DT_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/DT_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:10.html
serialized_ex

### Wine dataset

In [26]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/DT_wine.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:1.html


serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/DT_wine_instance:291_num_feat

serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/DT_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/DT_wine_instance:725_num_features:8_feat

serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/DT_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/DT_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/DT_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/DT_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/DT_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/DT_wine_instance:725_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/DT_wine_instance:725_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:7.html
serialized_e

### Diabetes dataset

In [27]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/DT_diabetes.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:d

serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:4

serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:False_kernel_s:6.html
serialized_explanati

serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:auto_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/DT_diabetes_instance:86_num_features:11_feat_selec:auto_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/DT_diabet

## KNN

### Heart dataset

In [28]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

# model_file_name = 'serialized_model/KNN_heart.pkl'
model_file_name = 'serialized_model/KNN_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sa

serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_expl

serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/KNN_heart_instanc

serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/KNN_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/KNN_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/KNN_hear

### Wine dataset

In [29]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/KNN_wine.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s

serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/KNN_wine_instance:291

serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/KNN_wine_instance:725_num_features:8_feat_sel

serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/KNN_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/KNN_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/KNN_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/KNN_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/KNN_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:7.html
serialized_expla

### Diabetes dataset

In [30]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/KNN_diabetes.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_

serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartil

serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_ke

serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/KNN_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:9.html

## Gaussian Process

### Heart dataset

In [31]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/GaussianProcessClassifier_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/GaussianProcessClassifier_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:143_num_features:13_feat_se

serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/G

serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:none_discre:quartile_

serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_heart_instance:96_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_

serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/GaussianProcessClassifier_heart_instance:242_num_features:13_feat_selec:none_discre:decile_sai:True_ke

### Wine dataset

In [32]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/GaussianProcessClassifier_wine.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/GaussianProcessClassifier_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:291_num_features:8_feat_selec:lasso_path

serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/GaussianProcessClassifier_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:3.html
serialized_explanat

serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:3.html
serial

serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/GaussianProcessClassifier_wine_instance:755_num_features:8_feat_selec:lasso_path_discre:quartile_s

serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/GaussianProcessClassifier_wine_instance:60_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/GaussianProces

### Diabetes dataset

In [33]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/GaussianProcessClassifier_diabetes.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/GaussianProcessClassifier_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:86_num_featu

serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:none_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:auto_discre:quartile_sai:Fal

serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:auto_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1003_num_

serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:none_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:auto_discre:quartile_sai:True_k

serialized_explanation/GaussianProcessClassifier_diabetes_instance:1458_num_features:11_feat_selec:auto_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:1458_num_features:11_feat_selec:auto_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/GaussianProcessClassifier_diabetes_instance:791_num_features:11_

## SVM

### Heart dataset

In [34]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/svm_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sa

serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_expl

serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/svm_heart_instanc

serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:none_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:auto_discre:quartile_sai:False_kernel_s:10.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:True_kernel_s:10.html
serialized_explanation/svm_heart_instance:143_num_features:13_feat_selec:auto_discre:decile_sai:False_kernel_s:10.html
serialized_explanation/svm_heart_instance:203_num_features:13_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:10.html
serialized_explanation/svm_hear

### Wine dataset

In [35]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/svm_wine.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:1.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:1.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:1.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:1.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s

serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:3.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:3.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:3.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:3.html
serialized_explanation/svm_wine_instance:291

serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:none_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:none_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:5.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:5.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:5.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:5.html
serialized_explanation/svm_wine_instance:725_num_features:8_feat_sel

serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:auto_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/svm_wine_instance:291_num_features:8_feat_selec:auto_discre:decile_sai:False_kernel_s:7.html
serialized_explanation/svm_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:7.html
serialized_explanation/svm_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:7.html
serialized_explanation/svm_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:7.html
serialized_explanation/svm_wine_instance:725_num_features:8_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:7.html
serialized_expla

### Diabetes dataset

In [36]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/svm_diabetes.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['doente', 'saudável'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}_instance:{2}_""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}_feat_selec:{1}_discre:{2}_""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}_kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:2.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:2.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:2.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:2.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_

serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:4.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:4.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:4.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:4.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartil

serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:6.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:6.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:6.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:6.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_ke

serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:forward_selection_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:True_kernel_s:9.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:lasso_path_discre:decile_sai:False_kernel_s:9.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:True_kernel_s:9.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:none_discre:quartile_sai:False_kernel_s:9.html
serialized_explanation/svm_diabetes_instance:86_num_features:11_feat_selec:none_discre:decile_sai:True_kernel_s:9.html

# SHAP

In [37]:
# import sklearn
# import shap
# from sklearn.model_selection import train_test_split

# # print the JS visualization code to the notebook
# shap.initjs()

# # train a SVM classifier
# X_train,X_test,Y_train,Y_test = train_test_split(*shap.datasets.iris(), test_size=0.2, random_state=0)
# svm = sklearn.svm.SVC(kernel='rbf', probability=True)
# svm.fit(X_train, Y_train)

# # use Kernel SHAP to explain test set predictions
# explainer = shap.KernelExplainer(svm.predict_proba, X_train, link="logit")
# shap_values = explainer.shap_values(X_test, nsamples=100)

# # plot the SHAP values for the Setosa output of the first instance
# shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_test.iloc[0,:], link="logit")


In [38]:
# shap.force_plot(explainer.expected_value[0], shap_values[0], X_test, link="logit")

In [39]:
# shap.summary_plot(shap_values, X_test.iloc[0:1000,:])

# Meu exemplo